In [1]:

# HIGH PRIORITY:

# IMPORTANT
# - Does it run from scratch on a new computer?
# - Add saved buffers/trajs to the real repo
# - change num_rollouts back


# Add preloaded
# Actually collect and check [15]

# STRETCH
# - Setup.py
# - Collect val set
# - partial observability
# - log more often
# - Add early stopping on training
# - maybe add training error bar?
# - Add a reset
# - Final summary
# - Intro text


# Setup - do this once

In [2]:
!git clone https://github.com/aliengirlliv/teachable 1> /dev/null

fatal: destination path 'teachable' already exists and is not an empty directory.


In [3]:
cd teachable

/home/olivia/Documents/Teachable/demo/teachable


In [4]:
!git checkout sac 1> /dev/null
!pip install -r reqs.txt 1> /dev/null

Already on 'sac'


# Setup - Do this each time you reload the notebook

In [5]:
%matplotlib tk

In [6]:
cd teachable

[Errno 2] No such file or directory: 'teachable'
/home/olivia/Documents/Teachable/demo/teachable


In [7]:
from final_demo import *
import ipywidgets as widgets
from IPython.display import HTML

In [8]:
cd teachable

[Errno 2] No such file or directory: 'teachable'
/home/olivia/Documents/Teachable/demo/teachable


# Setup

## Instructions
1. Select the collection mode.
    - "Advice" runs the "Improvement" phase of our method, allowing you to coach an agent through
    - "Demos" lets you collect trajectories providing actions each timestep.
    - "Precollected" loads a set of trajectories precollected using 1/2 hr of human time using our method.
2. Select a save name (any string describing this experiment).
3. Collect demos below


In [9]:
widgets.Box(items)

Box(children=(ToggleButtons(description='Collect Mode', options=('Advice', 'Demos', 'Precollected'), value='Ad…

# Collection
To collect data, run the block below. A window will open which lets you collect data.  

## Task
The agent's task is to unlock a door by collecting a matching-colored key and using it on the corresponding door. (To speed up training time, we always spawn the agent in the same room as the locked door.)

### Using Advice

The agent you will be coaching has been pretrained to understand Waypoint advice. It has never seen this particular environment/task before, and has never had to unlock a door. Click on a square to tell the agent to head there and manipulate any item present.

### Providing Demos
Use the arrow keys to navigate, Page Up/Down to manipulate objects, and Space to open doors.

### Using precollected data
Skip the block below


In [13]:
collector = HumanFeedback(collect_type=env_params.collect_type, 
                          save_path=env_params.save_path)

# Train

Here, we train an advice-free policy on the collected trajectories using the buffer of collected trajectories.


In [14]:
args = make_args(collector, env_params.save_path)  
run_experiment(args)

Logging to /home/olivia/Documents/Teachable/demo/teachable/logs/temp
Done on timestep 200 0
Video saved at /home/olivia/Documents/Teachable/demo/teachable/logs/temp/vid.avi
Finished rollouts, acc = 0.125, success = 0.0
Finished with success: 0.0, stoch acc: 0.125, det acc: 0.0
Finetuning achieved success: 0.0, stoch acc: 0.125

 ---------------- Iteration 0 ----------------
Obtaining samples...
Distilling ...
/home/olivia/Documents/Teachable/demo/teachable/logs/temp
--------------------------------------
| BufferSize              | 1        |
| Distill/Accuracy_Train  | 1        |
| Distill/Accuracy_Val    | 1        |
| Distill/Entropy_Train   | 0.00549  |
| Distill/Entropy_Val     | 0.00545  |
| Distill/Label_Accura... | 0        |
| Distill/Loss_Train      | 0.000535 |
| Distill/Loss_Val        | 0.000531 |
| Distill/Mean_Dist_Train | -1       |
| Distill/Mean_Dist_Val   | -1       |
| Distill/Std_Train       | -1       |
| Distill/Std_Val         | -1       |
| Distill/TotalLoss_Tr

KeyboardInterrupt: 

# Visualize

Play video saved during training. This agent does not receive advice.

In [15]:
html_str = display_trained_model(env_params.save_path)
HTML(html_str)

FFmpeg Script Ran Successfully
